In [1]:
# t-SNE
from sklearn.manifold import TSNE
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
# manifold learning; latent (space) representation

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
# scaling -> 0~1 범위 안이 되도록
X_train_full = X_train_full.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255

In [4]:
#tsne = TSNE()

#X_2D = tsne.fit_transform(X_test.reshape((-1, 28*28)))

In [5]:
#X_2D.shape

In [6]:
#_, ax = plt.subplots()
#ax.scatter(X_2D[:, 0], X_2D[:, 1], c=y_test, s=1)
#plt.show()

In [7]:
# Autoencoder
# encoder단이랑 decoder단을 별도로 만들기
# 만들어두고 한 Sequential로 합치기
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

In [27]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(42)
# keras.models.Sequential이랑 같음
encoder = keras.Sequential([
    keras.layers.Reshape([28, 28, 1]),
    # 합성곱 층 만들기
    # 점차 차원 수를 줄여서
    keras.layers.Conv2D(filters=16, kernel_size=3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(filters=30, kernel_size=3, padding="same", activation="relu"),
    keras.layers.GlobalAvgPool2D()

    # ....
    # 5 by 5, 32개 activation map
])

decoder = tf.keras.Sequential([
    keras.layers.Dense(3 * 3 * 16),
    keras.layers.Reshape((3, 3, 16)), # 다시 "이미지"로
    keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu"), # 9 by 9
    keras.layers.Conv2DTranspose(16, 3, strides=2, padding="same", activation="relu"), # 22 by 22
    keras.layers.Conv2DTranspose(1, 3, strides=2, padding="same"),
    # encoder에서의 kernel_size가 역순으로 decoder에서 kernel_size로 들어가도록
    # encoder에서의 pooling pool_size가 역순으로 decoder에서 stride로 들어가도록
    keras.layers.Reshape([28, 28])
])

ae = keras.Sequential([encoder, decoder])
# reconstruction loss라는 이름의 손실 함수가 있는게 아니고
# 얼마나 잘 복원(reconstruction) 했는지 정도를 나타내는 지표가 여럿 있음
# 예를 들면 MSE로 input과 reconstructed input 사이의 Sqaured Error를 구한다든지, Cross Entropy를 구한다든지..
ae.compile(loss="mse", optimizer="adam")

history = ae.fit(X_train, X_train, epochs=10, validation_data=(X_valid, X_valid))

Epoch 1/10
 662/1719 [==========>...................] - ETA: 43s - loss: 0.0476

KeyboardInterrupt: ignored